# Общие функции и константы для всех блоков модели

In [68]:
import sqlalchemy as sa
import warnings
from os import path, getcwd
warnings.filterwarnings('ignore')

strYearDBPath=path.join('..','DB', 'year.sqlite3') # путь к рабочей базе данных фактических рядов SQLite 
strExYearDBPath=path.join('..','DB', 'exog_year.sqlite3') # путь к рабочей базе данных экзогенных переменных SQLite
strExParamDBPath=path.join('..','DB', 'exog_param.sqlite3') # путь к рабочей базе данных экзогенных параметров SQLite
strSvodDBPath=path.join('..','DB', 'svod.sqlite3') # путь к рабочей базе данных итоговых рядов SQLite 

# константы базы данных SQLite3
strDatas_table='datas' # название таблицы с данными в базе данных SQLite
strHeaders_table='headers' # название таблицы с текстовыми и кодовыми именами рядов в базе данных SQLite


import matplotlib.pyplot as plt

In [56]:
def make_SELECT_YEAR_string(rowCodes, datas_table=strDatas_table, header_table=strHeaders_table):
    assert type(rowCodes) in (str, list, tuple), 'make_SELECT_YEAR_string from() COMMON module: error in param type'
    strSelect='''select {datas_table}.date, {datas_table}.value, {header_table}.code2 from {datas_table} join {header_table}
                  on {datas_table}.code={header_table}.code {where_condiiton}'''
    
    
    if type(rowCodes)==str:
        return strSelect.format(where_condiiton = 'where {header_table}.code2=="{code2}"'.format(header_table=header_table, code2=rowCodes),
                               datas_table=datas_table, header_table=header_table)
    elif type(rowCodes) in (tuple, list):
        return strSelect.format(where_condiiton = 'where {header_table}.code2 in {codes2}'.format(header_table=header_table, 
                                                                                                  codes2=tuple(rowCodes)),
                               datas_table=datas_table, header_table=header_table)
    return None

def make_frame(pdf):
    k=pdf.set_index(['date', 'code2']).unstack().reset_index().set_index('date')
    k.columns=[c[1] for c in k.columns]
    return k

def scale(pdf, list_fields=[], multiplier=1):
    pdf[list_fields] *= multiplier
    return pdf

def add(pdf, list_fields=[], param=0):
    pdf[list_fields] += param
    return pdf

## Ниже - тесты функций. Чтобы работало, нужно убрать магию

In [67]:
%%script false --no-raise-error

import pandas as pd

strDB_path=path.join('..', 'DB', 'bankrp.sqlite3')

strYearDBPath=path.join('..','DB', 'year.sqlite3') # путь к рабочей базе данных фактических рядов SQLite 
strExYearDBPath=path.join('..','DB', 'exog_year.sqlite3') # путь к рабочей базе данных экзогенных переменных SQLite
strExParamDBPath=path.join('..','DB', 'exog_param.sqlite3') # путь к рабочей базе данных экзогенных параметров SQLite
strSvodDBPath=path.join('..','DB', 'svod.sqlite3') # путь к рабочей базе данных итоговых рядов SQLite 


# ------------

conWork = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=strYearDBPath)) # connection к рабочей базе данных

pdf=(pd.read_sql(make_SELECT_YEAR_string(['GDP_x', 'GDP_Iq','CPIAv','KeyRate','Unmpl_s']), con=conWork)
     .pipe(make_frame)
     .pipe(scale, list_fields=['GDP_x', 'GDP_Iq'], multiplier=100)
     .pipe(add, 'Unmpl_s', -100))
pdf